# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [85]:
# imports

import os
import requests
import json
import ollama
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [86]:
# constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

    
MODEL = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

API key found and looks good so far!


In [87]:
# set up environment

class Website:
    """
    A simple utility class to represent a scraped website, including its title, text, and links.
    """

    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    def __init__(self, url):
        self.url = url
        self.title = "No title found"
        self.text = ""
        self.links = []

        self._fetch()

    def _fetch(self):
        response = requests.get(self.url, headers=self.HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')

        if soup.title:
            self.title = soup.title.string.strip()

        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)

        self.links = [link.get('href') for link in soup.find_all('a', href=True)]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n"


In [69]:
system_prompt = "You are a technical expert that can take users questions and provides responses\
                 Provide responses in a clear, simple, explanative and polite way\
                 Respond in markdown"


In [75]:
def get_responses_user_prompt(question):
    user_prompt = f"Ask a question that needs technical explanation "
    user_prompt += "Respond in technical terms.\n"
    user_prompt += "Links (some might be relative links):\n"
    #user_prompt += "\n".join(website.links)
    return user_prompt

In [59]:
# here is the question; type over this to ask something new

def ask_technical_question(question):
    response = openai.chat.completions.create(
    model= MODEL,  
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Question: {question}\n Answer:"}
        ],
    max_tokens=200,  # Limit the response length
    temperature=0.5  # Control the creativity of the response
    )
# Correct indentation
    answer = response.choices[0].message.content
    return answer

# Example usage

print(ask_technical_question(question))
#create a system message to tell the model its answering a technical question.

question = """
 Please explain what https and ssh mean and why it is used:
 yield from {book.get("author") for book in books if book.get("author")}
"""
print(ask_technical_question(question))

TCP (Transmission Control Protocol) and UDP (User Datagram Protocol) are two fundamental protocols used for data transmission over the Internet.

### TCP (Transmission Control Protocol)
- **Connection-Oriented**: Establishes a connection before data transfer and ensures reliable communication.
- **Reliability**: Guarantees that data packets are delivered in order and without errors. If packets are lost, TCP will retransmit them.
- **Flow Control**: Manages data flow to prevent overwhelming the receiver.
- **Use Cases**: Ideal for applications where reliability is crucial, such as web browsing (HTTP/HTTPS), email (SMTP), and file transfers (FTP).

### UDP (User Datagram Protocol)
- **Connectionless**: Sends data without establishing a connection, which makes it faster but less reliable.
- **No Guarantees**: Does not ensure packet delivery, order, or integrity. Packets may be lost or arrive out of order.
- **Low Latency**: Suitable for applications where speed is
**HTTPS (Hypertext Trans

In [81]:
# Get gpt-4o-mini to answer, with streaming

# Send the question to the OpenAI model and get the response

def multishot(question):
    # Define previous interactions and system prompt for multi-shot prompting
    previous_interactions = [
        {"role": "system", "content": "You are a fun and cheerful technical expert "},
        
        # Example of a prior user request (can be expanded with more interactions)
        {"role": "user", "content": "Can you help me answer this technical question?"},  
        
        {"role": "assistant", "content": "Of course! I'd be happy to help you with that. Could you please provide the question and context?"},

        {"role": "user", "content": "What is wrong with this code? This is the error message I get"},

        {"role": "assistant", "content": "Got it! Thank you. Let me check the error and your code."},

        # You can include more prior examples or questions here
    ]

    # Add the current user prompt as the next interaction
    previous_interactions.append(
        {"role": "user", "content": question}
    )

    # Stream the user response and display back to the user

    stream = openai.chat.completions.create(
    model= MODEL,  
    messages= previous_interactions,
    stream=True,
    max_tokens=200,  # Limit the response length
    temperature=0.5  # Control the creativity of the response
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Loop through the stream and build the response incrementally
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

question = input("Please enter your technical question: ")
multishot(question)

Please enter your technical question:  SSH vs HTTPS


Ah, the classic SSH vs. HTTPS debate! Both are used for secure communication, but they serve different purposes and have distinct characteristics. Let’s break it down:

### SSH (Secure Shell)
- **Purpose**: Primarily used for secure remote login and command execution on servers. It's also used for secure file transfers (like SCP and SFTP).
- **Authentication**: Typically uses public/private key pairs for authentication. You can also use passwords, but keys are more secure.
- **Port**: Default port is 22.
- **Use Case**: Ideal for developers and administrators who need to access servers securely and perform operations remotely.

### HTTPS (Hypertext Transfer Protocol Secure)
- **Purpose**: Used to secure communications over a computer network, primarily the internet. It encrypts data transmitted between a web browser and a server.
- **Authentication**: Relies on SSL/TLS certificates to authenticate the server and establish a secure connection.
- **Port**: Default port is

In [92]:
# Get Llama 3.2 to answer



def multishot(question):
    # Define previous interactions and system prompt for multi-shot prompting
    previous_interactions = [
        {"role": "system", "content": "You are a fun and cheerful technical expert "},
        
        # Example of a prior user request (can be expanded with more interactions)
        {"role": "user", "content": "Can you help me answer this technical question?"},  
        
        {"role": "assistant", "content": "Of course! I'd be happy to help you with that. Could you please provide the question and context?"},

        {"role": "user", "content": "What is wrong with this code? This is the error message I get"},

        {"role": "assistant", "content": "Got it! Thank you. Let me check the error and your code."},

        # You can include more prior examples or questions here
    ]

    # Add the current user prompt as the next interaction
    previous_interactions.append(
        {"role": "user", "content": question}
    )

    # Stream the user response and display back to the user

    stream = ollama.chat(
    model= MODEL_LLAMA,  
    messages= previous_interactions,
    stream=True,
    options={
        "num_predict":200,  # Limit the response length
        "temperature":0.5  # Control the creativity of the response
       }
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    # Loop through the stream and build the response incrementally
    for chunk in stream:
        if chunk.message and chunk.message.content:
           response += chunk.message.content
           response = response.replace("```", "").replace("markdown", "")
           update_display(Markdown(response), display_id=display_handle.display_id)

question = input("Please enter your technical question: ")
multishot(question)

Please enter your technical question:  http vs ssh


HTTP (Hypertext Transfer Protocol) and SSH (Secure Shell) are two different protocols used for transferring data over the internet.

**HTTP**

HTTP is a request-response protocol that uses TCP/IP to transfer data between a client (usually a web browser) and a server. It's commonly used for:

* Web browsing
* File transfers
* Email

**Security concerns:**

1. **Insecure by default**: HTTP is not encrypted by default, making it vulnerable to eavesdropping, tampering, and man-in-the-middle attacks.
2. **Data exposure**: HTTP requests can be intercepted and read by unauthorized parties.

**SSH**

SSH is a secure communication protocol that uses encryption (specifically, the Transport Layer Security/Secure Sockets Layer protocol) to securely transfer data between two endpoints over an insecure network.

**Security benefits:**

1. **Encryption**: SSH encrypts all data transmitted, ensuring confidentiality, integrity, and authenticity.
2. **Authentication**: SSH provides secure